# **MODELAGEM E ESTIMATIVA DOS PARÂMETROS CINÉTICOS PARA PRODUÇÃO DE AMOXILINA**
### ***POR SÍNTESE ENZIMÁTICA EM BIORREATOR BATELADA E BATELADA ALIMENTADA***

In [17]:
#%pip install openpyxl
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random
import scipy
import glob
import os

### Importando dados

In [49]:
def import_dfs():
    # Path to the folder containing CSV files
    folder_path = '../dados/'

    # Get a list of all CSV files in the folder
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    # Initialize an empty list to hold dataframes
    data_frames = []

    # Read the first CSV file to establish the schema
    schema = ['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm','apa_t']


    # Load each remaining CSV file, reorder columns, and append to the list
    for file in csv_files:
        df = pd.read_csv(file)  
        print(file)
        df = df[schema]  # Reorder columns to match the schema
        data_frames.append(df)
    
    csv_files = [name.split('\\')[-1].rstrip('.csv') for name in csv_files]
    return data_frames,csv_files 
data_frames,file_list = import_dfs()
print(data_frames[0].columns)
print(file_list)

../dados\25.45nh60ab20.csv
../dados\25.46nh30ab30.csv
../dados\25.47nh5ab80.csv
../dados\5.100nh40ab80.csv
../dados\5.102nh20ab40.csv
../dados\5.103nh40ab30.csv
../dados\5.106nh100ab40.csv
../dados\5.107nh55ab55.csv
../dados\5.109nh5ab55.csv
../dados\5.110nh10ab55.csv
../dados\5.112nh78ab35.csv
../dados\5.48nh20ab80.csv
../dados\5.80nh12ab40.csv
../dados\5.81nh5ab30.csv
../dados\nh60ab80.csv
Index(['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm', 'apa_t'], dtype='object')
['25.45nh60ab20', '25.46nh30ab30', '25.47nh5ab80', '5.100nh40ab80', '5.102nh20ab40', '5.103nh40ab30', '5.106nh100ab40', '5.107nh55ab55', '5.109nh5ab55', '5.110nh10ab55', '5.112nh78ab35', '5.48nh20ab80', '5.80nh12ab40', '5.81nh5ab30', 'nh60ab80']


### Model

In [19]:
def enzymic_amox(t,y, 
kcat1,
kcat2,
Km1,
Km2,  
Tmax, 
Ken,  
kAB,  
kAN,  
kAOH, 
kNH):
    FAB = 0
    FNH = 0 
    
    CAB = y[0]
    CAN = y[1]
    CNH = y[2]
    CAOH = y[3]

    Cez = 1

    # Consumo de ester
    VAB = (kcat1*CAB*Cez)/((Km1*(1 + (CAN/kAN) + (CAOH/kAOH))) + CAB)
    
    # Hidrolise de amoxicilina
    VAN = (kcat2*CAN*Cez)/((Km2*(1 + (CAB/kAB) + (CNH/kNH) + (CAOH/kAOH))) + CAN)
    
    # Enzima saturada com 6-apa
    X   = CNH/(Ken + CNH)
    
    # Sintese enzimatica
    VS  = VAB*Tmax*X

    # Hidrolise de ester
    Vh1 = (VAB - VS) 

    dy = np.zeros(4)

    # C. ester
    dy[0] = ((-(VS - VAN) - (Vh1 + VAN)) + FAB) 
    
    # C. amox
    dy[1] = (VS - VAN)                         
    
    # C. 6-apa
    dy[2] = (-(VS - VAN) + FNH)                
    
    # C. POHPG
    dy[3] =  (Vh1 + VAN)
    
    return np.array(dy)      

In [20]:
def ode15s_amox(P, CI, t):
    return scipy.integrate.solve_ivp(enzymic_amox,t_span=(t[0],t[-1]),t_eval=t,y0=CI,method='BDF',args=P).y.T

In [103]:
kcat1        = 0.181
kcat2        = 0.395
Km1          = 5.449
Km2          = 1.694
Tmax         = 0.824
Ken          = 7.947
kAB          = 0.682
kAN          = 1.989
kAOH         = 9.856
kNH          = 9.763

P = np.zeros(10)
P[0]   = kcat1    
P[1]   = kcat2    
P[2]   = Km1      
P[3]   = Km2      
P[4]   = Tmax     
P[5]   = Ken      
P[6]   = kAB      
P[7]   = kAN      
P[8]   = kAOH     
P[9]  = kNH 

Np = len(P)

def mcmc(df,P,name,priori,N=5000,ode_solver=ode15s_amox):

    Np = len(P)
    # Extraindo dados
    CAB_medida  = df.loc[:,'ester_mm'].dropna().to_numpy()
    CAN_medida = df.loc[:,'amox_mm'].dropna().to_numpy()
    CNH_medida = df.loc[:,'apa_mm'].dropna().to_numpy()
    CAOH_medida = df.loc[:,'aoh_mm'].dropna().to_numpy()
    t = df.loc[:,'apa_t'].dropna().to_numpy()
    experiments = [CAB_medida,CAN_medida,CNH_medida,CAOH_medida]

    Cez = 1

    CI = np.array([CAB_medida[0],CAN_medida[0],CNH_medida[0],CAOH_medida[0]])

    Y = ode_solver(P,CI,t)
    CAB_ref   =  Y[:,0]      
    CAN_ref   =  Y[:,1]    
    CNH_ref   =  Y[:,2]      
    CAOH_ref  =  Y[:,3]
    desvio_CAB =  0.1*max(CAB_ref) 
    desvio_CAN =  0.1*max(CAN_ref)
    desvio_CNH =  0.1*max(CNH_ref)
    desvio_CAOH = 0.1*max(CAOH_ref)
    
    waux      = 6e-3 
    media_g   = 1       #Média gaussiana
    desviop_g = 0.6     #Desvio dos parâmetros ao utilizar priori gaussian

    estimate = [i for i in range(10)]
    Nfix = len(estimate)

    p_ref = P                   
    p_old = P                     

    media_MCMC = media_g*P
    desvio_P   = desviop_g*P

    w = np.ones((1,Np))   
    w = w*waux  

    parametro_exato = (p_ref*np.ones((N,1))).T
    
    aceitacao    = np.zeros((1,N))  
    cadeia       = np.zeros((Np,N)) 
    conv_likeli  = np.zeros((1,N))
    k=0


    Lk_1 = np.dot((CAB_medida - CAB_ref),(CAB_medida - CAB_ref).T) / (desvio_CAB**2)     
    Lk_2 = np.dot((CAN_medida - CAN_ref),(CAN_medida - CAN_ref).T) / (desvio_CAN**2)      
    Lk_3 = np.dot((CNH_medida - CNH_ref),(CNH_medida - CNH_ref).T) / (desvio_CNH**2)       
    Lk_4 = np.dot((CAOH_medida - CAOH_ref),(CAOH_medida - CAOH_ref).T) / (desvio_CAOH**2)  
    
    lk_old = Lk_1 + Lk_2 + Lk_3 + Lk_4
    prior_old = np.sum((((p_old - media_MCMC))/((desvio_P)**2)))
    cadeia[:,0] = p_old
    
    # Contador para verificar a aceitação
    for i in range(0, N):  # Python index starts at 0, MATLAB at 1
        progress = (i / N) * 100

        # New parameter vector
        P_new = p_old + w * np.random.randn(Np) * p_old
        #print('p_new: ', P_new)
        # Simulate using ODE solver
        Y = ode_solver(P_new[0], CI,t)

        CAB  = Y[:,0]
        CAN  = Y[:,1]        
        CNH  = Y[:,2]
        CAOH = Y[:,3]

        # Calculate new prior
        Prior_new = np.sum(((P_new - media_MCMC) / desvio_P) ** 2)

        # Likelihood calculations
        Lk_1 = np.dot((CAB_medida - CAB),(CAB_medida - CAB).T) / (desvio_CAB**2)        
        Lk_2 = np.dot((CAN_medida - CAN),(CAN_medida - CAN).T) / (desvio_CAN**2)         
        Lk_3 = np.dot((CNH_medida - CNH),(CNH_medida - CNH).T) / (desvio_CNH**2)        
        Lk_4 = np.dot((CAOH_medida - CAOH),(CAOH_medida - CAOH).T) / (desvio_CAOH**2)   
    
        Lk_new = Lk_1 + Lk_2 + Lk_3 + Lk_4
        
        # MCMC acceptance check
        if np.log(random.random()) < (-0.5 * (Lk_new + Prior_new - lk_old - prior_old)):
            p_old = P_new
            lk_old = Lk_new
            prior_old = Prior_new
            k += 1

        # Store results
        aceitacao[0,i] = k
        cadeia[:, i] = p_old
        conv_likeli[0,i] = lk_old + prior_old
    
    fig,ax = plt.subplots(2,5,figsize=[20,8])
    ax = ax.flatten()
    labels = ['kcat1',
    'kcat2',
    'Km1',
    'Km2',  
    'Tmax', 
    'Ken',  
    'kAB',  
    'kAN',  
    'kAOH', 
    'kNH']
    for idx,g in enumerate(ax):
        g.plot(cadeia[idx,:])    
        g.set_title(labels[idx])
        g.set_ylabel("Value")
        g.set_xlabel("Iteration")
    fig.savefig(f"./chains/{priori}_{N}_{name}.png",dpi=400)
    plt.close()    

    aq = int(0.9 * N)  # Starting index for the burn-in period
    IC = 0.99          # Confidence interval level

    # Confidence interval bounds
    xaux = (1 - IC) / 2
    IC_inf = xaux
    IC_sup = 1 - xaux
    estimate = [i for i in range(Np)]

    amostra = N - aq + 1

    AB_amostra = np.zeros((amostra, len(t)))
    AN_amostra = np.zeros((amostra, len(t)))
    NH_amostra = np.zeros((amostra, len(t)))
    AOH_amostra = np.zeros((amostra, len(t)))

    output_par = np.zeros((amostra,Np))
    for idx,i in enumerate(range(aq, N + 1)):
        ii = i - aq
        paux = cadeia[:, i - 1]
        output_par[ii,:] = paux
        Y = ode_solver(paux,CI,t)
        CAB  = Y[:,0]
        CAN  = Y[:,1]
        CNH  = Y[:,2]
        CAOH = Y[:,3]
        
        AB_amostra[ii, :] = CAB
        AN_amostra[ii, :] = CAN
        NH_amostra[ii, :] = CNH
        AOH_amostra[ii, :] = CAOH 

    AB_media = np.zeros((len(t)))
    AN_media = np.zeros((len(t)))
    NH_media = np.zeros((len(t)))
    AOH_media = np.zeros((len(t)))
    AB_sup = np.zeros((len(t)))
    AN_sup = np.zeros((len(t)))
    NH_sup = np.zeros((len(t)))
    AOH_sup = np.zeros((len(t)))
    AB_inf = np.zeros((len(t)))
    AN_inf = np.zeros((len(t)))
    NH_inf = np.zeros((len(t)))
    AOH_inf = np.zeros((len(t)))   
    
    for i in range(len(t)):
        AB_media[i] = np.mean(AB_amostra[:, i])
        y = np.percentile(AB_amostra[:, i], [IC_inf * 100, IC_sup * 100])
        AB_inf[i] = y[0]
        AB_sup[i] = y[1]

        NH_media[i] = np.mean(NH_amostra[:, i])
        y = np.percentile(NH_amostra[:, i], [IC_inf * 100, IC_sup * 100])
        NH_inf[i] = y[0]
        NH_sup[i] = y[1]

    for i in range(len(t)):
        AN_media[i] = np.mean(AN_amostra[:, i])
        y = np.percentile(AN_amostra[:, i], [IC_inf * 100, IC_sup * 100])
        AN_inf[i] = y[0]
        AN_sup[i] = y[1]

        AOH_media[i] = np.mean(AOH_amostra[:, i])
        y = np.percentile(AOH_amostra[:, i], [IC_inf * 100, IC_sup * 100])
        AOH_inf[i] = y[0]
        AOH_sup[i] = y[1]

    # individual plot
    # Ensure 'results' directory exists
    results_dir = 'results_mcmc/'

    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    names = ['POH-PGME','Amoxicillin','6-APA','POHPG']
    
    style = ['or','og','ob','ok']
    mstyle = ['-r','-g','-b','-k']
    medidas = [
        [AB_media, AB_inf, AB_sup],
        [AN_media, AN_inf, AN_sup],
        [NH_media, NH_inf, NH_sup],
        [AOH_media,AOH_inf,AOH_sup]
    ]
    fig,ax = plt.subplots(2,2,figsize=(10,10))
    for idx,axis in enumerate(ax.flatten()):
        sname = names[idx]
        medida = medidas[idx]
        axis.plot(t, experiments[idx], style[idx], label=sname)
        axis.plot(t, medida[0], mstyle[idx])
        axis.plot(t, medida[1], '--r',linewidth=0.5)
        axis.plot(t, medida[2], '--r',linewidth=0.5) 
        axis.set_xlabel('Tempo (min)')
        axis.set_ylabel(f'Concentração de {sname} (mm)')  # Use f-string for proper string formatting
        axis.legend()  # Adjust legend position
    fig.savefig(os.path.join(results_dir, f"{priori}_{N}_{name}_mcmc.png"), dpi=300)
    plt.close()  # Close the figure to release memory
    return output_par
        


In [74]:
pd.DataFrame(mcmc(data_frames[2],P=P,name='test',N=100)).size

110

In [96]:
# luci parameters

kcat1        = 0.178 #Constante catal�tica do consumo do �ster (�mol/i.u. per min)
 
kcat2        = 0.327 #Constante catal�tica da hidr�lise da amoxicilina (�mol/i.u. per min)
 
Km1          = 7.905 #Constante de Michaelis-Menten ou constante de afinidade para consumo do �ster(mM) 
 
Km2          = 12.509 #Constante de Michaelis-Menten ou constante de afinidade para hidr�lise da amoxicilina(mM)
 
Tmax         = 0.606 #Taxa de convers�o m�xima do complexo acil-enzima-n�cleo em produto
 
Ken          = 14.350 #Constante de adsor��o do 6-APA
 
kAB          = 3.78 #Constante de inibi��o do �ster(POHPGME)(mM)
 
kAN          = 9.174 #Constante de inibi��o da amoxicilina (mM)
 
kAOH         = 10.907 #Constante de inibi��o do POHPG, produto da hidr�lise da amoxicilina (mM)
 
kNH          = 62.044 #Constante de inibi��o do 6-APA

luci_P = np.zeros(10)
luci_P[0]   = kcat1    
luci_P[1]   = kcat2    
luci_P[2]   = Km1      
luci_P[3]   = Km2      
luci_P[4]   = Tmax     
luci_P[5]   = Ken      
luci_P[6]   = kAB      
luci_P[7]   = kAN      
luci_P[8]   = kAOH     
luci_P[9]  = kNH 

In [97]:
kcat1        = 0.181 #Constante catal�tica do consumo do �ster (�mol/i.u. per min)
 
kcat2        = 0.395 #Constante catal�tica da hidr�lise da amoxicilina (�mol/i.u. per min)
 
Km1          = 5.449 #Constante de Michaelis-Menten ou constante de afinidade para consumo do �ster(mM) 
 
Km2          = 1.694 #Constante de Michaelis-Menten ou constante de afinidade para hidr�lise da amoxicilina(mM)
 
Tmax         = 0.824 #Taxa de convers�o m�xima do complexo acil-enzima-n�cleo em produto
 
Ken          = 7.947 #Constante de adsor��o do 6-APA
 
kAB          = 0.682 #Constante de inibi��o do �ster(POHPGME)(mM)
 
kAN          = 1.989 #Constante de inibi��o da amoxicilina (mM)
 
kAOH         = 9.856 #Constante de inibi��o do POHPG, produto da hidr�lise da amoxicilina (mM)
 
kNH          = 9.763 #Constante de inibi��o do 6-APA

lucas_P = np.zeros(10)
lucas_P[0]   = kcat1    
lucas_P[1]   = kcat2    
lucas_P[2]   = Km1      
lucas_P[3]   = Km2      
lucas_P[4]   = Tmax     
lucas_P[5]   = Ken      
lucas_P[6]   = kAB      
lucas_P[7]   = kAN      
lucas_P[8]   = kAOH     
lucas_P[9]  = kNH 

In [98]:
kcat1        = 0.59 #Constante catal�tica do consumo do �ster (�mol/i.u. per min)
 
kcat2        = 3.07 #Constante catal�tica da hidr�lise da amoxicilina (�mol/i.u. per min)
 
Km1          = 0.2 #Constante de Michaelis-Menten ou constante de afinidade para consumo do �ster(mM) 
 
Km2          = 27.47 #Constante de Michaelis-Menten ou constante de afinidade para hidr�lise da amoxicilina(mM)
 
Tmax         = 0.96 #Taxa de convers�o m�xima do complexo acil-enzima-n�cleo em produto
 
Ken          = 16.03 #Constante de adsor��o do 6-APA
 
kAB          = 2672.04 #Constante de inibi��o do �ster(POHPGME)(mM)
 
kAN          = 4.59 #Constante de inibi��o da amoxicilina (mM)
 
kAOH         = 4.51 #Constante de inibi��o do POHPG, produto da hidr�lise da amoxicilina (mM)
 
kNH          = 4550.28 #Constante de inibi��o do 6-APA

process_P = np.zeros(10)
process_P[0]   = kcat1    
process_P[1]   = kcat2    
process_P[2]   = Km1      
process_P[3]   = Km2      
process_P[4]   = Tmax     
process_P[5]   = Ken      
process_P[6]   = kAB      
process_P[7]   = kAN      
process_P[8]   = kAOH
process_P[9]   = kNH

In [102]:
optimized_par = np.array([1.76111242e-01, 4.28768542e-01, 1.71836876e+00, 1.42996053e-02, 9.53931641e-01, 1.82350367e+01, 3.45380244e-03, 4.84532693e-01, 2.32604795e+01, 2.39272396e+00])

In [106]:

all_priori = [luci_P,lucas_P,optimized_par,process_P]
all_priori_names = ['luci','lucas','bestnm','article']

for priori_name,priori in zip(all_priori_names,all_priori):
    labels = ['kcat1',
        'kcat2',
        'Km1',
        'Km2',  
        'Tmax', 
        'Ken',  
        'kAB',  
        'kAN',  
        'kAOH', 
        'kNH']

    all_par_df = pd.DataFrame(np.zeros((len(data_frames),len(labels))),columns=labels)
    all_parstderror_df = pd.DataFrame(np.zeros((len(data_frames),len(labels))),columns=labels)

    N = 100000
    Priori_name = priori_name

    count=0
    for df,name in zip(data_frames,file_list):
        print(f'Currently: {name}')
        Ps = mcmc(df,P=priori,name=name,priori=Priori_name,N=N)
        np.savez_compressed(f'./results_mcmc/pickles/{N}_{Priori_name}_{name}',Ps)
        P_conf = pd.DataFrame(Ps)
        P_mean = P_conf.describe().iloc[1,:].to_numpy()
        P_error = P_conf.describe().iloc[2,:].to_numpy() / np.sqrt(P_conf.size)
        for i in range(len(labels)):
            print(f'{labels[i]}: {P_mean[i]} +- {P_error[i]}')
        all_par_df.iloc[count,:] = P_mean
        all_parstderror_df.iloc[count,:] = P_error
        count += 1

    all_par_df.to_csv(f'./results_mcmc/{N}_{Priori_name}_mean.csv')
    all_parstderror_df.to_csv(f'./results_mcmc/{N}_{Priori_name}_error.csv')

Currently: 25.45nh60ab20
kcat1: 0.015436850953349231 +- 6.387824829682369e-06
kcat2: 0.035160312762031444 +- 3.143833586235767e-05
Km1: 1.3887710516466527 +- 0.0006514114841570899
Km2: 3.292768603323298 +- 0.001371575973476445
Tmax: 0.29827949236678863 +- 0.00010265809545799865
Ken: 5.793180975966734 +- 0.003082423923505071
kAB: 1.0868139717167784 +- 0.0014156917965464652
kAN: 0.7288789901215875 +- 0.0007093876487819869
kAOH: 6.824288209908932 +- 0.004040024279880847
kNH: 72.06877588876712 +- 0.04832652670630044
Currently: 25.46nh30ab30
kcat1: 0.08617783086114156 +- 3.2612431929379496e-05
kcat2: 0.4659305695270026 +- 0.0002293112801546174
Km1: 1.8426450320204315 +- 0.0011235796560220951
Km2: 6.17540965185368 +- 0.0025731394317467186
Tmax: 0.7789647063386984 +- 0.00045955724169033683
Ken: 16.18131276167889 +- 0.010850539020874045
kAB: 0.5595323622545562 +- 0.0006990159934176073
kAN: 5.076927234797279 +- 0.0024846262940521
kAOH: 0.5646311700379594 +- 0.0004080106360849647
kNH: 13.3599944

c:\Users\artur\Desktop\lneoc\pibic_lneoc\deep_learning\venv\Lib\site-packages\scipy\integrate\_ivp\bdf.py:416: RuntimeWarning: invalid value encountered in subtract
  D[order + 2] = d - D[order + 1]


kcat1: 0.27645673951865973 +- 9.647640962422469e-05
kcat2: 0.33664216749640447 +- 0.000174706396549978
Km1: 7.847783156523919 +- 0.003310498685640405
Km2: 2.0447176713521027 +- 0.0010894220172229965
Tmax: 0.16878387938164285 +- 8.326221176949203e-05
Ken: 2.2348873531910303 +- 0.0009321728380547257
kAB: 1.966401789186744 +- 0.0006629290237816611
kAN: 0.783245842244664 +- 0.00039878751403229514
kAOH: 3.4412722078164513 +- 0.0022412486038656605
kNH: 28.319602278550903 +- 0.01716209226832138
Currently: nh60ab80


c:\Users\artur\Desktop\lneoc\pibic_lneoc\deep_learning\venv\Lib\site-packages\scipy\integrate\_ivp\bdf.py:416: RuntimeWarning: invalid value encountered in subtract
  D[order + 2] = d - D[order + 1]


kcat1: 0.25422547441506377 +- 0.0001162672435866995
kcat2: 0.4070291513649137 +- 0.0001544101189435339
Km1: 13.849625675451872 +- 0.0059745649752452
Km2: 13.975676438767431 +- 0.0036286461554079353
Tmax: 0.7764479470771098 +- 0.0001461276399818728
Ken: 9.699648353437906 +- 0.004852896702486451
kAB: 2.742107914013138 +- 0.001351974166929607
kAN: 2.941614460259211 +- 0.0019162196986469723
kAOH: 17.965762808060834 +- 0.0066130010123148205
kNH: 55.24900456302106 +- 0.054193866614215436
Currently: 25.45nh60ab20
kcat1: 0.013993127137728054 +- 5.03524806206153e-06
kcat2: 0.4488753550167556 +- 0.00020170733988752302
Km1: 0.06841264616788663 +- 5.630638685299138e-05
Km2: 0.4216653913178091 +- 0.00026691502208426974
Tmax: 0.30625055194487255 +- 8.414268744717847e-05
Ken: 3.9161829060745696 +- 0.002707547348601039
kAB: 0.01886014225029629 +- 6.938796230757327e-06
kAN: 1.5637392997483284 +- 0.0009300194985728317
kAOH: 0.13456099496204663 +- 8.518110606556579e-05
kNH: 0.5178322918324648 +- 0.000658

c:\Users\artur\Desktop\lneoc\pibic_lneoc\deep_learning\venv\Lib\site-packages\scipy\integrate\_ivp\bdf.py:416: RuntimeWarning: invalid value encountered in subtract
  D[order + 2] = d - D[order + 1]


kcat1: 0.1517539680415325 +- 4.261290317155652e-05
kcat2: 0.04810162855752461 +- 2.4909623540659923e-05
Km1: 1.2736921303935034 +- 0.000572271039493284
Km2: 0.0002865633972025132 +- 1.1248698698442891e-07
Tmax: 1.5167208025201455 +- 0.0006978950760567861
Ken: 23.329481600970947 +- 0.010248126514683075
kAB: 0.005795036624364039 +- 5.614090800103519e-06
kAN: 0.07339671302358892 +- 7.115527701447332e-05
kAOH: 18.31740749635493 +- 0.012338352978562134
kNH: 1.2071099237038139 +- 0.0007752021965004404
Currently: nh60ab80
kcat1: 0.18923169189441855 +- 5.320278708135658e-05
kcat2: 0.11884539161786116 +- 5.8004495564834823e-05
Km1: 0.7869043690097834 +- 0.0007510826286817301
Km2: 0.003543236130454947 +- 1.75276350769025e-06
Tmax: 1.036569127457188 +- 0.0003215315993872053
Ken: 24.980169951626863 +- 0.010384224416531746
kAB: 0.0029829656840147556 +- 1.9153490820497107e-06
kAN: 0.24361871828712758 +- 0.00021284509672278926
kAOH: 1.1887504968605338 +- 0.0013019488014205474
kNH: 0.5139389563706739 

In [82]:
all_par_df

,kcat1,kcat2,Km1,Km2,Tmax,Ken,kAB,kAN,kAOH,kNH
0,0.158518,0.401212,5.719994,1.549992,0.750419,8.199490,0.691457,1.922903,9.172399,9.154323
1,0.153147,0.375674,5.914852,1.770082,0.818282,8.023208,0.660718,1.946860,9.274640,9.457907
2,0.208032,0.423901,5.251847,1.673897,0.708887,8.155987,0.678776,1.992368,9.922135,9.706939
3,0.161904,0.381966,5.054066,1.536224,0.716063,8.671082,0.730297,1.932140,10.232565,9.214589
4,0.177908,0.423334,5.279244,1.566771,0.724330,7.855454,0.732915,1.938074,9.489092,10.098933
5,0.176745,0.363003,5.468201,1.648033,0.805911,8.014404,0.750682,1.856920,9.733972,10.286278
6,0.161056,0.376579,5.414072,1.765328,0.897848,8.236451,0.642307,1.982582,9.761364,9.909059
7,0.179075,0.384435,6.622670,1.665928,0.763479,8.441335,0.678881,1.817192,9.788823,10.147161
8,0.215017,0.396739,5.436745,1.608624,0.751684,8.907679,0.684905,1.971795,9.357324,9.831989
9,0.207188,0.432025,5.512091,1.604432,0.779720,7.963021,0.722699,2.149931,9.452302,9.855040


In [83]:
all_parstderror_df

,kcat1,kcat2,Km1,Km2,Tmax,Ken,kAB,kAN,kAOH,kNH
0,0.000050,0.000107,0.003347,0.000245,0.000233,0.003943,0.000117,0.000715,0.004296,0.002344
1,0.000058,0.000174,0.006294,0.001552,0.000650,0.002832,0.000235,0.000645,0.006527,0.003832
2,0.000073,0.000404,0.002007,0.001441,0.000372,0.007186,0.000524,0.001382,0.007727,0.004210
3,0.000123,0.000626,0.002434,0.001149,0.000400,0.004649,0.000270,0.003333,0.011082,0.007354
4,0.000122,0.000445,0.002071,0.000411,0.000193,0.003165,0.000368,0.001143,0.008207,0.002882
5,0.000082,0.000303,0.002852,0.000952,0.000665,0.003949,0.000843,0.001428,0.011633,0.002493
6,0.000074,0.000108,0.001558,0.000824,0.000303,0.014401,0.000342,0.001095,0.007622,0.005415
7,0.000146,0.000211,0.007201,0.001511,0.000480,0.007062,0.000636,0.001558,0.004636,0.008320
8,0.000314,0.000107,0.006052,0.000896,0.000436,0.005432,0.000349,0.001071,0.007050,0.009206
9,0.000062,0.000135,0.002459,0.001059,0.000652,0.005166,0.000225,0.001560,0.013622,0.005199
